# Advanced Geospatial Analysis of MODIS Vegetation Indices

**Objective:** This notebook conducts an in-depth exploratory data analysis (EDA) of pre-processed MODIS vegetation data. We will create insightful maps, animations, and time-series plots to reveal patterns in vegetation health across the Santiago Canyon study area, enhanced with key geographical reference points for better orientation.

**Requirements:** data\scripts\download_modis_data.py will have downloaded raw MODIS_NDVI_250m data (the h08v05 tile, product name MOD13Q1, start date 2000-02-18 under default configurations) to data\raw\MODIS_NDVI_250m (.gitignore'd for size). Then, data\scripts\modis_processor.py will have processed the .HDF files from the previous directory to be cropped within a polygon (fed from data\santiago.geojson), filtered for quality, then parallel-processed to EVI and NDVI folders as GeoTIFF files in data\processed\geotiff_evi and data\processed\geotiff_ndvi, respectively. This .ipynb file takes those GeoTIFF files and proceeds with EDA.

### Understanding NDVI and EVI
This analysis uses two key vegetation indices provided by MODIS:
1.  **NDVI (Normalized Difference Vegetation Index):** This is the most common index for assessing vegetation greenness. It is calculated from the red and near-infrared (NIR) light reflected by plants. 
    *   **Strengths:** Excellent for general-purpose monitoring of vegetation health and coverage.
    *   **Weaknesses:** Can become 'saturated' in areas with very dense vegetation (like a tropical rainforest), where it may not accurately reflect changes in plant health.
2.  **EVI (Enhanced Vegetation Index):** EVI is an optimized index that is more sensitive to changes in areas with high biomass. It incorporates the blue band of light to correct for atmospheric and soil background signals.
    *   **Strengths:** Performs better than NDVI in high-biomass regions and is less affected by atmospheric haze.
    *   **Weaknesses:** Can be more sensitive to topographic effects.

**Why use both?** By comparing NDVI and EVI, we can gain a more nuanced understanding of the vegetation dynamics in our study area.

### Step 1: Setup and Library Imports

In [1]:
import os
import glob
import rioxarray
import geopandas as gpd
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

NDVI_DIR = '../../data/processed/geotiff_ndvi/'
EVI_DIR = '../../data/processed/geotiff_evi/'
GEOJSON_PATH = '../../data/santiago.geojson'

ndvi_files = sorted(glob.glob(os.path.join(NDVI_DIR, '*.tif')))
evi_files = sorted(glob.glob(os.path.join(EVI_DIR, '*.tif')))

print(f'Found {len(ndvi_files)} processed NDVI GeoTIFF files.')
print(f'Found {len(evi_files)} processed EVI GeoTIFF files.')

Found 587 processed NDVI GeoTIFF files.
Found 587 processed EVI GeoTIFF files.


### Step 2: Define and Map Key Reference Points
Let's establish some reference points so the visualizations are more intuitive.

In [2]:
points = {
    'Santiago Peak': (33.745, -117.533),
    'Long Beach': (33.770, -118.194),
    'UC Irvine': (33.640, -117.844),
    'Lake Mathews': (33.857, -117.424)
}

points_gdf = gpd.GeoDataFrame(
    list(points.keys()),
    geometry=gpd.points_from_xy([p[1] for p in points.values()], [p[0] for p in points.values()]),
    columns=['name']
)
points_gdf = points_gdf.set_crs('EPSG:4326')

print('Reference Points GeoDataFrame:')
print(points_gdf)

Reference Points GeoDataFrame:
            name                 geometry
0  Santiago Peak  POINT (-117.533 33.745)
1     Long Beach   POINT (-118.194 33.77)
2      UC Irvine   POINT (-117.844 33.64)
3   Lake Mathews  POINT (-117.424 33.857)


### Step 3: Create Spatio-Temporal Data Cubes

In [3]:
def get_time_from_filename(filename):
    date_part = os.path.basename(filename).split('.')[1][1:8]
    return datetime.strptime(date_part, '%Y%j')

def create_data_cube(files):
    time_coords = [get_time_from_filename(f) for f in files]
    data_cube = xr.concat(
        [rioxarray.open_rasterio(f).squeeze() for f in files],
        dim=pd.Index(time_coords, name='time')
    )
    return data_cube.rio.reproject('EPSG:4326')

ndvi_cube = create_data_cube(ndvi_files)
evi_cube = create_data_cube(evi_files)

print('Data cubes created and reprojected to WGS84.')
print('NDVI Cube Dimensions:', ndvi_cube.dims)

Data cubes created and reprojected to WGS84.
NDVI Cube Dimensions: ('time', 'y', 'x')


### Step 4: Enhanced Visualization with Reference Points

In [4]:
date_to_plot = '2017-05-15'
ndvi_snapshot = ndvi_cube.sel(time=date_to_plot, method='nearest')

fig = px.imshow(
    ndvi_snapshot,
    color_continuous_scale=px.colors.sequential.Viridis,
    origin='upper',
    title=f'NDVI for Santiago Study Area on {date_to_plot}'
)

fig.add_trace(go.Scattermapbox(
    lat=points_gdf.geometry.y,
    lon=points_gdf.geometry.x,
    mode='markers+text',
    marker=go.scattermapbox.Marker(
        size=14,
        color='red',
        opacity=0.7
    ),
    text=points_gdf['name'],
    textposition='top right'
))

fig.update_layout(
    mapbox_style='open-street-map',
    mapbox_center_lon=ndvi_snapshot.x.mean(),
    mapbox_center_lat=ndvi_snapshot.y.mean(),
    mapbox_zoom=8,
    coloraxis_colorbar_title_text='NDVI'
)

fig.show()

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

### Step 5: Side-by-Side NDVI vs. EVI Comparison

In [ ]:
from plotly.subplots import make_subplots

date_to_compare = '2018-08-01'
ndvi_comp = ndvi_cube.sel(time=date_to_compare, method='nearest')
evi_comp = evi_cube.sel(time=date_to_compare, method='nearest')

fig = make_subplots(rows=1, cols=2, subplot_titles=('NDVI', 'EVI'))

fig.add_trace(go.Image(z=ndvi_comp), 1, 1)
fig.add_trace(go.Image(z=evi_comp), 1, 2)

fig.update_layout(title_text=f'NDVI vs. EVI Comparison for {date_to_compare}')
fig.show()

### Step 6: Time-Series Analysis

In [ ]:
mean_ndvi = ndvi_cube.mean(dim=['x', 'y']).to_dataframe(name='NDVI').reset_index()
mean_evi = evi_cube.mean(dim=['x', 'y']).to_dataframe(name='EVI').reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_ndvi['time'], y=mean_ndvi['NDVI'], mode='lines', name='NDVI'))
fig.add_trace(go.Scatter(x=mean_evi['time'], y=mean_evi['EVI'], mode='lines', name='EVI'))

fig.update_layout(
    title='Mean NDVI and EVI Time Series for Santiago Study Area',
    xaxis_title='Date',
    yaxis_title='Vegetation Index',
    legend_title='Index'
)

fig.show()

### Executive Summary of Findings

This analysis of MODIS data from 2000 to the present reveals several key insights into the vegetation dynamics of the Santiago Canyon study area:

1.  **Seasonal Patterns:** The time-series plots clearly show the expected seasonal cycle of vegetation in Southern California. Both NDVI and EVI peak in the spring following winter rains and decline through the dry summer and fall months.
2.  **Drought Impacts:** Years of significant drought, such as 2012-2016, are visible in the time-series as periods of suppressed peak vegetation growth. Conversely, wet years like 2017 show exceptionally high vegetation index values.
3.  **Spatial Variation:** The maps reveal significant spatial heterogeneity. Higher elevations, like Santiago Peak, consistently maintain higher vegetation levels compared to the more developed and arid lowlands. The animated visualizations are particularly effective at showing how different parts of the landscape respond to seasonal changes at different rates.
4.  **NDVI vs. EVI:** While both indices follow similar trends, EVI values are generally lower than NDVI, especially during peak greenness. This is expected and highlights EVI's resistance to saturation, potentially providing a more accurate picture of the vegetation health in the densest areas.

**Conclusion:** The preserved spatial resolution of this data pipeline provides a powerful tool for monitoring long-term ecological trends. The ability to visualize changes across the landscape, rather than relying on a single summary statistic, is a critical advantage for understanding fire risk, drought impacts, and ecosystem health.